In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

# Normalizacja wartości pikseli do [0,1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Dodawanie wymiaru (wymagane przez framework do treningu CNN)
x_train = np.expand_dims(x_train, -1)  # (60000, 28, 28) -> (60000, 28, 28, 1)
x_test = np.expand_dims(x_test, -1)  # (10000, 28, 28) -> (10000, 28, 28, 1)

print(x_train.shape, x_test.shape)

In [ ]:
# Dodawanie losowego szumu do obrazów
noise_factor = 0.5

x_train_noisy = x_train + noise_factor * np.random.normal(size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(size=x_test.shape)

# Przycinanie wartości do zakresu [0, 1]
x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)

In [ ]:
# Budowa prostego modelu autoencoder
autoencoder = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),

    layers.Conv2D(16, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2, padding="same"),
    layers.Conv2D(8, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2, padding="same"),

    layers.Conv2D(8, 3, activation="relu", padding="same"),
    layers.UpSampling2D(2),
    layers.Conv2D(16, 3, activation="relu", padding="same"),
    layers.UpSampling2D(2),
    layers.Conv2D(1, 3, activation="sigmoid", padding="same")
])

autoencoder.compile(
    optimizer="adam",
    loss="binary_crossentropy"
)

autoencoder.summary()

In [ ]:
autoencoder.fit(
    x_train_noisy,
    x_train,
    epochs=5,
    batch_size=128,
    shuffle=True,
    validation_data=(x_test_noisy[10:], x_test[10:]) # jako, że nie mamy osobnego validation set, użyję prawie wszystkich obrazów z test set,
                                                     # pozostawiając pierwsze 10 na faktyczną predykcję (nie użyję ich do walidacji) - na potrzeby przykładu
                                                     # te 10 obrazów wystarczy, by obejrzeć ostateczne działanie autoencodera
)

In [ ]:
decoded_images = autoencoder.predict(x_test_noisy[:10])

plt.figure(figsize=(12, 4))

for i in range(10):
    ax = plt.subplot(3, 10, i + 1)
    plt.imshow(x_test_noisy[i].squeeze(), cmap="gray")
    plt.axis("off")
    if i == 0:
        ax.set_ylabel("Noisy")

    ax = plt.subplot(3, 10, i + 11)
    plt.imshow(decoded_images[i].squeeze(), cmap="gray")
    plt.axis("off")
    if i == 0:
        ax.set_ylabel("Denoised")

    ax = plt.subplot(3, 10, i + 21)
    plt.imshow(x_test[i].squeeze(), cmap="gray")
    plt.axis("off")
    if i == 0:
        ax.set_ylabel("Original")

plt.tight_layout()
plt.show()